In [1]:
import threading
from etl.etl_run import ETLRun

from models import ParentPair, ChildPair

from sqlalchemy import create_engine
from db_constants import Base
from etl.data_loader import DataLoader


engine = create_engine('sqlite:///Cryptos.db', echo=False)
Base.metadata.create_all(engine)

In [2]:
stop_event = threading.Event()
data_loader = DataLoader()
etl = ETLRun(stop_event, pairs_limit=25, data_loader=data_loader)
processor_thread = threading.Thread(target=data_loader.process_items)
etl.start_threads()
processor_thread.start()

try:
    # Keep the main thread running
    while True:
        pass
except KeyboardInterrupt:
    # On KeyboardInterrupt, stop the WebSocketThread
    stop_event.set()
    etl.stop_threads()
    data_loader.stop_queue_process()
    processor_thread.join()


/Users/mirbilal/Desktop/personal/tankx/etl/data_loader.py:35: SAWarning: Usage of the 'Session.add()' operation is not currently supported within the execution stage of the flush process. Results may not be consistent.  Consider using alternative event listeners or connection-level operations instead.
  self.session.add(parent_pair_obj)


Connected ETHPAX
Connected BTCTRYConnected BTCUSDP

Connected ETHBTC
Connected ETHBTC
Connected ETHGBP
Connected BTCUSDC
Connected BTCUAH
Connected ETHUSDT
Connected USDTPAX
Connected ETHTUSD
Connected ETHUST
Connected USDTTUSD
Connected ETHBTC
Connected ETHBTC
Connected ETHBTC
Connected ETHUSDC
Connected BTCGBP
Connected ETHBTC
Connected ETHBTC
Connected BTCUST
Connected ETHUSDP
Connected ETHBTC
Connected ETHBUSD
Connected BTCAEUR
Connected ETHPAX
Connected ETHUSDT
Connected ETHTUSD
Connected ETHFDUSD
Connected BTCUSDT
Connected ETHUAH
Connected BTCBUSD
Connected ETHTRY
Connected ETHEUR
Connected ETHPLN
Connected ETHBTC
Connected ETHUSDT
Connected ETHBTC
Connected BTCNGN
Connected ETHUSDT
Connected ETHBIDR
Connected ETHBTC
Connected ETHAEUR
Connected ETHBTC
Connected ETHBTC
Connected ETHAUD
Connected ETHBTC
Connected BTCTUSD
Connected ETHNGN
Connected ETHRUB
Connected USDTBTCConnected BTCFDUSD

Connected BTCPLN
Connected ETHBTC
Connected ETHBTC
Connected ETHDAI
Connected ETHBTC
Connec

AttributeError: 'DataLoader' object has no attribute 'stop_queue_proccess'

In [ ]:

parent_pairs = etl.extractor.pair_sockets
for parent_pair, sub_pairs in parent_pairs.items():
    for sub_pair, thread in sub_pairs.items():
        print(thread.updates)